For pre-processing the message data as input to the RNN

In [ ]:
import csv
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import re

In [ ]:
# Search for digits (ie user name or user id?)
digits = re.compile("\d")

Download your Facebook message data and put the messages.htm file in the same directory.

In [ ]:
# Read the html file
print("\nReading file...\n")
htm = urllib.request.urlopen("file:messages.htm").read()
print(htm[:100])

In [ ]:
print("\nMaking soup...\n")
soup = BeautifulSoup(htm, "lxml")
print(soup.title)

Format of the message data:


In [ ]:
# Get threads
print("Looking for threads")
Threads = soup.findAll(class_="thread")
print("\nFound ", len(Threads), " threads\n")

In [ ]:
# Read the name by the facebook address with the user ID
# 'user' variable is of the form '100001785489441@facebook.com'
# we extract the user id, open the facebook page of the user and get the
# real name
# Does not always work so we catch exceptions and print out to manually update
# Save them in a dict for quicker access

# Empty dict
user_dict = {}
# Load saved dict
user_dict = np.load('user_dict.npy').item()
def getUser(user_id):
    if user_id in user_dict:
        return user_dict[user_id]
    try:
        fb = urllib.request.urlopen("https://www.facebook.com/" + user_id).read()
        fb_soup = BeautifulSoup(fb, "lxml")
        user = fb_soup.title.getText().split("|")[0]
        # Corner case for getting "security check" rather than user name
        if "Security" in user: 
            print("Could not extract user's name: https://www.facebook.com/" + user_id)
            user = input()
            user_dict[user_id] = user
            return user
        user_dict[user_id] = user
        return user
    except:
        print("Could not extract user's name: https://www.facebook.com/" + user_id)
        user = input()
        user_dict[user_id] = user
        return user

In [ ]:
# Write all messages in each thread to a new file
i = 1
total_extracted = 0
LENGTH_FOR_THREAD = 50 # Only conversations over a certain number of messages
DATE_FOR_THREAD = 2017 # Only threads with recent activity are considered
DATE_FOR_MESSAGE = 2015 # Only more recent messages within considered threads are kept
for thread in Threads:
    messages = thread.findAll(class_="message_header")
    # Check most recent activity
    if int(messages[0].find(class_="meta").getText().split(',')[1].split()[2]) < DATE_FOR_THREAD:
        continue
    # Check conversation length
    if len(messages) < LENGTH_FOR_THREAD:
        continue
    print("="*125)
    print("\nThread #",i)
    print(len(messages), " messages\n")
    message_path = "./messages/messages-"+str(i)+".txt"
    with open(message_path, "w", encoding="utf-8") as file:
        for j,message_header in enumerate(messages):
            text = message_header.find_next("p").getText()
            user = message_header.find_next(class_="user").getText()
            meta = message_header.find(class_="meta").getText()
            # Check message age
            if int(meta.split(',')[1].split()[2]) < DATE_FOR_THREAD:
                continue
            total_extracted += 1
            if not user or digits.search(user):
                user_id = user.split('@')[0]
                user = getUser(user_id)
            
            if text!=None and user!=None and meta!=None:
                #file.write(meta)
                file.write("\n"+user)
                file.write("\n\t"+text+"\n")
            else:
                print ("\nCouldn't write row\n")
                print ("\n", message_header, "\n")
            # Print for progress
            if j < 5:
                #print(meta)
                print(user)
                print("\t",text,"\n")
    print("Written to ", message_path)
    i+=1
print("\n\n Completed pre-processing {} messages extracted.\n".format(total_extracted))
# Save user dict to file
np.save("user_dict.npy", user_dict)